### Can we classify each phase as relatively low or high anxiety for each subject? ###

In [41]:
# IMPORTING MODULES
import glob
import importlib
import matplotlib.pyplot as plt
import numpy as np
import os
cvx_path = os.path.abspath(os.path.join('..', '..', 'cvxEDA', 'src'))
module_path = os.path.abspath(os.path.join('..', '..', 'src'))
import pandas as pd
import random
import scipy.signal as ss
import shap
import sys
sys.path.append(module_path)

import tools.data_reader_apd as dr
import tools.display_tools as dt
import tools.preprocessing as preprocessing
import train

import lightgbm as lgb
from lightgbm import LGBMClassifier
from scipy.fft import fft, fftfreq, fftshift
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, \
    mean_absolute_error, mean_squared_error, log_loss
from sklearn.model_selection import train_test_split, cross_validate, RepeatedKFold
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import normalize
from xgboost import XGBClassifier

import cvxopt.solvers
cvxopt.solvers.options['show_progress'] = False

import warnings
warnings.filterwarnings(
    "ignore", 
    category=RuntimeWarning
)

temp_a, _ = train.Train_APD.get_apd_data_ranking([train.Metrics.BPM], phases=dr.Phases.PHASES_LIST, normalize=False)
idx = temp_a[temp_a["bpm"] > 200].index 
invalid_apd_subjects = set(temp_a["subject"].iloc[idx].tolist())
idx = temp_a[temp_a["bpm"] < 40].index 
invalid_apd_subjects.update(set(temp_a["subject"].iloc[idx].tolist()))

In [42]:
importlib.reload(train)

models = {
    "SVM": SVC(),
    "LGB": LGBMClassifier(),
    "RF": RandomForestClassifier(random_state=16),
    "XGB": XGBClassifier(random_state=16),
    # "random": None
}

parameters = {
    "SVM": [{
        "kernel": ["rbf", "poly", "sigmoid"],
        "C": [0.1, 1, 10, 100],
        "gamma": [1, 0.1, 0.01, 0.001, "scale", "auto"],
        "probability": [True]
    }],
    "LGB": [{
        "objective": ["binary"],
        "num_leaves": [10, 20, 30, 40, 50],
        "max_depth": [3, 4, 5, 6, 7],
        "metric": ["mean_absolute_error", "mean_squared_error", "binary_logloss"]
    }],
    "RF": [{
        "n_estimators": [10, 20, 30, 40, 50],
        "max_features": ["sqrt", "0.4"],
        "min_samples_split": [3, 4, 5, 6, 7],
        "random_state": [16]
    }],
    "XGB": [{
        "objective": ["binary:logistic"],
        "learning_rate": [0.01, 0.1, 0.3, 0.5],
        "max_depth": [4, 5, 6, 7],
        "n_estimators": [10, 20, 30, 40, 50],
        "eval_metric": ["error"],
        "use_label_encoder": [False],
        "random_state": [16]
    }],
    # "random": None
}

metrics = [
    train.Metrics.BPM, 
    train.Metrics.RMSSD, 
    train.Metrics.HF_RR, 
    train.Metrics.LF_RR, 
    train.Metrics.SDNN, 
    train.Metrics.MEAN_SCL, 
    train.Metrics.SCR_RATE, 
# ]
] + train.Metrics.STATISTICAL

In [45]:
# K-FOLD CROSS-VALIDATION FOR HYPERPARAMETER SELECTION
importlib.reload(train)
importlib.reload(dr)
importlib.reload(dt)


random.seed(16)

model_phases = [
    "Baseline_Rest", 
    "BugBox_Relax", "BugBox_Anticipate", "BugBox_Exposure", "BugBox_Break",
    "Speech_Relax", "Speech_Anticipate", "Speech_Exposure", "Speech_Break"
]

threshold = "fixed"

anxiety_label_type = None
x, y = train.Train_APD.get_apd_data_ranking(
    metrics, model_phases, verbose=False, anxiety_label_type=anxiety_label_type, 
    threshold=threshold, normalize=True, combine_phases=False, standardize=False
)
x = x.drop(["phaseId"], axis=1)
inds = pd.isnull(x).any(axis=1).to_numpy().nonzero()[0]
x = x.drop(labels=inds, axis=0).reset_index(drop=True)
y = y.drop(labels=inds, axis=0).reset_index(drop=True)

# drop subjects with noisy data
# x = x[~x["subject"].isin(invalid_apd_subjects)].reset_index(drop=True)
# y = y[~y["subject"].isin(invalid_apd_subjects)].reset_index(drop=True)

if anxiety_label_type is not None:
    x.drop(labels=["anxietyGroup"], axis=1)
    
acc_results = {
    "SVM": [],
    "LGB": [],
    "RF": [],
    "XGB": [],
    # "random": []
}
reports = {
    "SVM": [],
    "LGB": [],
    "RF": [],
    "XGB": [],
    # "random": []
}
best_models = {}
ensemble_weights = {}

num_iters = 5
get_importance = True
for _ in range(num_iters):
    # HYPERPARAMETER TUNING
    model_data = train.grid_search_cv(
        models, parameters, x, y, by_subject=True, save_metrics=True, is_resample=True,
        get_importance=get_importance, drop_subject=True, test_size=0.1, folds=5
    )

    for model_name in models.keys():
        best_models[model_name] = model_data[model_name]["best_model"]
        print(f"{model_name}: {model_data[model_name]['best_params']}")

    # # FEATURE SELECTION
    x_train, y_train = model_data["train"]
    features = {name: metrics+["lf_hf_ratio"] for name in models.keys()}
    # features = train.feature_selection(best_models, model_data["cv"], x_train, y_train, n_features=5)

    # TEST USING OPTIMIZED MODELS AND FEATURES
    x_test, y_test = model_data["test"]
    out = train.train_test_model(best_models, features, x_train, y_train, x_test, y_test)

    for model_name in acc_results:
        acc_results[model_name].append(out[model_name]["performance"][0])
        reports[model_name].append(out[model_name]["performance"][1])
        # if get_importance:
            # try:
                # print("")
                # feature_imp = list(zip(metrics + ["lf_hf_ratio"], out[model_name]["performance"][2]))
                # feature_imp = sorted(feature_imp, key=lambda x: x[1], reverse=True)
                # print(feature_imp)
            # except Exception as e:
                # print(out[model_name]["performance"][2])
            # print("")

for model_name in acc_results.keys():
    print(f"Model evaluation metrics for {model_name}:")
    for i in range(len(reports[model_name])):
        report = reports[model_name][i]
        acc = acc_results[model_name][i]
        p = report["precision"]
        r = report["recall"]
        f1 = report["f1"]
        auc = report["auc"]
        ensemble_weights[model_name] = acc
        print(f"\tAccuracy: {acc}\n\tPrecision: {p}\n\tRecall: {r}\n\tF1-score: {f1}\n\tAUC score: {auc}\n" + "-"*40)
    print(f"Mean acc: {np.mean([acc_results[model_name][i] for i in range(len(reports[model_name]))])}")
    print(f"Mean F1-score: {np.mean([reports[model_name][i]['f1'] for i in range(len(reports[model_name]))])}")
    print(f"Mean AUC score: {np.mean([reports[model_name][i]['auc'] for i in range(len(reports[model_name]))])}")
    print("\n")

Grid search for SVM ...
Grid search for LGB ...
Grid search for RF ...


One or more of the test scores are non-finite: [0.56149407 0.57200715 0.58291904 0.58461609 0.57697986 0.55651561
 0.55518935 0.57178691 0.57207572 0.57527566 0.58465872 0.58275557
 0.59721952 0.59235044 0.59327669 0.56047158 0.56895977 0.5786141
 0.58399947 0.58692935 0.56588654 0.56791621 0.57066498 0.57967873
 0.58523948        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan]


Grid search for XGB ...
SVM: {'C': 0.1, 'gamma': 0.001, 'kernel': 'rbf', 'probability': True}
LGB: {'max_depth': 3, 'metric': 'mean_absolute_error', 'num_leaves': 10, 'objective': 'binary'}
RF: {'max_features': 'sqrt', 'min_samples_split': 5, 'n_estimators': 30, 'random_state': 16}
XGB: {'eval_metric': 'error', 'learning_rate': 0.1, 'max_depth': 5, 'n_estimators': 10, 'objective': 'binary:logistic', 'random_state': 16, 'use_label_encoder': False}
Training SVM ...
Model SVM, Actual: [0 1], [60 55], Predictions: [0 1], [107   8]
coef_ only available for SVC with linear kernel
Training LGB ...
Model LGB, Actual: [0 1], [60 55], Predictions: [0 1], [83 32]
Training RF ...
Model RF, Actual: [0 1], [60 55], Predictions: [0 1], [82 33]
Training XGB ...
Model XGB, Actual: [0 1], [60 55], Predictions: [0 1], [89 26]
Grid search for SVM ...
Grid search for LGB ...
Grid search for RF ...


One or more of the test scores are non-finite: [0.50985857 0.51958447 0.52588027 0.53607545 0.53148078 0.50526197
 0.52738018 0.52526801 0.53698079 0.54173455 0.51694332 0.530485
 0.52563246 0.53723844 0.53869974 0.56627827 0.54801582 0.54640403
 0.54704633 0.54076603 0.54127965 0.53039863 0.5241472  0.52860636
 0.52333246        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan]


Grid search for XGB ...
SVM: {'C': 10, 'gamma': 0.1, 'kernel': 'rbf', 'probability': True}
LGB: {'max_depth': 7, 'metric': 'mean_absolute_error', 'num_leaves': 30, 'objective': 'binary'}
RF: {'max_features': 'sqrt', 'min_samples_split': 6, 'n_estimators': 10, 'random_state': 16}
XGB: {'eval_metric': 'error', 'learning_rate': 0.01, 'max_depth': 5, 'n_estimators': 40, 'objective': 'binary:logistic', 'random_state': 16, 'use_label_encoder': False}
Training SVM ...
Model SVM, Actual: [0 1], [78 42], Predictions: [0 1], [94 26]
coef_ only available for SVC with linear kernel
Training LGB ...
Model LGB, Actual: [0 1], [78 42], Predictions: [0 1], [97 23]
Training RF ...
Model RF, Actual: [0 1], [78 42], Predictions: [0 1], [92 28]
Training XGB ...
Model XGB, Actual: [0 1], [78 42], Predictions: [0 1], [116   4]
Grid search for SVM ...
Grid search for LGB ...
Grid search for RF ...


...

Grid search for XGB ...
SVM: {'C': 0.1, 'gamma': 'scale', 'kernel': 'sigmoid', 'probability': True}
LGB: {'max_depth': 5, 'metric': 'mean_absolute_error', 'num_leaves': 10, 'objective': 'binary'}
RF: {'max_features': 'sqrt', 'min_samples_split': 3, 'n_estimators': 50, 'random_state': 16}
XGB: {'eval_metric': 'error', 'learning_rate': 0.3, 'max_depth': 7, 'n_estimators': 20, 'objective': 'binary:logistic', 'random_state': 16, 'use_label_encoder': False}
Training SVM ...
Model SVM, Actual: [0 1], [52 60], Predictions: [0 1], [107   5]
coef_ only available for SVC with linear kernel
Training LGB ...
Model LGB, Actual: [0 1], [52 60], Predictions: [0 1], [84 28]
Training RF ...
Model RF, Actual: [0 1], [52 60], Predictions: [0 1], [87 25]
Training XGB ...
Model XGB, Actual: [0 1], [52 60], Predictions: [0 1], [86 26]
Grid search for SVM ...
Grid search for LGB ...
Grid search for RF ...


One or more of the test scores are non-finite: [0.56135844 0.55993782 0.55721479 0.55941988 0.56391053 0.53049096
 0.56547914 0.57180648 0.57198178 0.56622965 0.53106095 0.53818544
 0.54165414 0.53077292 0.53437203 0.54871047 0.55365329 0.55901701
 0.55369372 0.55808211 0.53820811 0.53552118 0.54116695 0.54252867
 0.54619348        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan]


Grid search for XGB ...
SVM: {'C': 1, 'gamma': 1, 'kernel': 'rbf', 'probability': True}
LGB: {'max_depth': 5, 'metric': 'mean_absolute_error', 'num_leaves': 20, 'objective': 'binary'}
RF: {'max_features': 'sqrt', 'min_samples_split': 4, 'n_estimators': 40, 'random_state': 16}
XGB: {'eval_metric': 'error', 'learning_rate': 0.01, 'max_depth': 5, 'n_estimators': 30, 'objective': 'binary:logistic', 'random_state': 16, 'use_label_encoder': False}
Training SVM ...
Model SVM, Actual: [0 1], [48 64], Predictions: [0 1], [100  12]
coef_ only available for SVC with linear kernel
Training LGB ...
Model LGB, Actual: [0 1], [48 64], Predictions: [0 1], [75 37]
Training RF ...
Model RF, Actual: [0 1], [48 64], Predictions: [0 1], [86 26]
Training XGB ...
Model XGB, Actual: [0 1], [48 64], Predictions: [0 1], [100  12]
Grid search for SVM ...
Grid search for LGB ...
Grid search for RF ...


One or more of the test scores are non-finite: [0.556537   0.55885851 0.56678236 0.57121272 0.56943384 0.5353483
 0.5606447  0.58077395 0.58682994 0.58268709 0.55652857 0.56904256
 0.57731569 0.58632497 0.58819477 0.56971305 0.57903893 0.59136763
 0.59388563 0.60017642 0.57687236 0.59143776 0.59112807 0.59591061
 0.59550351        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan]


Grid search for XGB ...
SVM: {'C': 0.1, 'gamma': 0.001, 'kernel': 'sigmoid', 'probability': True}
LGB: {'max_depth': 6, 'metric': 'mean_absolute_error', 'num_leaves': 30, 'objective': 'binary'}
RF: {'max_features': 'sqrt', 'min_samples_split': 6, 'n_estimators': 50, 'random_state': 16}
XGB: {'eval_metric': 'error', 'learning_rate': 0.01, 'max_depth': 4, 'n_estimators': 50, 'objective': 'binary:logistic', 'random_state': 16, 'use_label_encoder': False}
Training SVM ...
Model SVM, Actual: [0 1], [75 33], Predictions: [0 1], [96 12]
coef_ only available for SVC with linear kernel
Training LGB ...
Model LGB, Actual: [0 1], [75 33], Predictions: [0 1], [56 52]
Training RF ...
Model RF, Actual: [0 1], [75 33], Predictions: [0 1], [73 35]
Training XGB ...
Model XGB, Actual: [0 1], [75 33], Predictions: [0 1], [95 13]
Model evaluation metrics for SVM:
	Accuracy: 0.5739130434782609
	Precision: 0.875
	Recall: 0.12727272727272726
	F1-score: 0.22222222222222224
	AUC score: 0.5553030303030303
-----

In [46]:
# ENSEMBLE
importlib.reload(train)
importlib.reload(dr)
importlib.reload(dt)

from scipy.stats import mode
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import f1_score, precision_score, recall_score, roc_auc_score


def train_predict_ensemble(ensemble_models, x_train, y_train, x_test, y_test, features, type="majority_vote", weights=None):
    y_preds = []
    if type == "majority_vote":
        # features = list(features.values())[0]
        # ensemble_models = [(key, ensemble_models[key]) for key in ensemble_models.keys()]
        # ensemble = VotingClassifier(estimators=ensemble_models, voting='hard', weights=weights)
        # ensemble.fit(x_train.loc[:, features], y_train)
        # y_preds = ensemble.predict(x_test.loc[:, features])
        for model_name in ensemble_models:
            x_test_temp = x_test.loc[:, features[model_name]]
            y_pred = ensemble_models[model_name].predict(x_test_temp)
            y_preds.append(y_pred)
        y_preds = mode(y_preds, axis=0)[0]
        y_preds = np.reshape(y_preds, (y_preds.shape[1], 1))
    elif type == "weighted_avg":
        features = list(features.values())[0]
        ensemble_models = [(key, ensemble_models[key]) for key in ensemble_models.keys()]
        ensemble = VotingClassifier(estimators=ensemble_models, voting='soft', weights=weights)
        ensemble.fit(x_train.loc[:, features], y_train)
        y_preds = ensemble.predict(x_test.loc[:, features])

    return y_preds


test_size = 0.15
folds = 5
ensemble_models = [
    "SVM", 
    "LGB",
    "RF",
    "XGB"
]
voting_type = "weighted_avg"

cv, x_train, y_train, x_test, y_test = train.kfold_train_test_split(x, y, test_size, is_resample=False, folds=folds, random_seed=19)
y_train = y_train.loc[:, "label"]
y_test = y_test.loc[:, "label"]

estimators = {name: best_models[name] for name in ensemble_models}
weights = [ensemble_weights[model_name] for model_name in estimators.keys()]
weights = np.divide(weights, np.sum(weights))
# weights = np.divide(weights, len(weights))
weights = [1 for _ in range(len(list(estimators.keys())))]
y_pred = train_predict_ensemble(estimators, x_train, y_train, x_test, y_test, features, type=voting_type, weights=weights)

acc = accuracy_score(y_test, y_pred)
p = precision_score(y_test, y_pred, zero_division=0)
r = recall_score(y_test, y_pred, zero_division=0)
f1 = f1_score(y_test, y_pred, zero_division=0)
auc = roc_auc_score(y_test, y_pred)
print(f"\tAccuracy: {acc}\n\tPrecision: {p}\n\tRecall: {r}\n\tF1-score: {f1}\n\tAUC score: {auc}\n" + "-"*40)

	Accuracy: 0.6510067114093959
	Precision: 0.7291666666666666
	Recall: 0.47297297297297297
	F1-score: 0.5737704918032787
	AUC score: 0.6498198198198198
----------------------------------------
